# Sérialisation - correction

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

### Exercice 1 : sérialisation d'un gros dataframe

**Etape 1 :** construction d'un gros dataframe composé de nombres aléatoires

In [ ]:
import random
values = [ [random.random() for i in range(0,20)] for _ in range(0,100000) ]
col = [ "col%d" % i for i in range(0,20) ]

In [ ]:
import pandas
df = pandas.DataFrame( values, columns = col )

**Etape 2 :** on sauve ce dataframe sous deux formats texte et sérialisé (binaire)

In [ ]:
df.to_csv("df_text.txt", sep="\t")

In [ ]:
df.to_pickle("df_text.bin")

**Etape 3 :** on mesure le temps de chargement

In [ ]:
%timeit pandas.read_csv("df_text.txt", sep="\t")

1 loops, best of 3: 995 ms per loop


In [ ]:
%timeit pandas.read_pickle("df_text.bin")

10 loops, best of 3: 18.5 ms per loop
